In [2]:
from ultralytics import YOLO, SAM
import cv2


yolo = YOLO("yolov8n.pt")  # load an official detection model
sam2 = SAM("Aerial_Vision_Analysis_Tool_v1.0/backend/model/sam2_b.pt")

# results = yolo.predict(source="0", show=True, classes=[0], conf=0.8)  # Display preds. Accepts all YOLO predict arguments

In [3]:
path_to_video = '/Users/luigi.greselin/Downloads/464248_pedestrians_walking_UK_200207London Streets14k014720p5000br.mp4'

# Experiment with poitn prompting

In [ ]:

cap = cv2.VideoCapture(path_to_video)
cap.set(3, 1280)
cap.set(4, 720)

output_video_path = "output_videos/processed_video.mp4"


fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))


while True:
    success, img = cap.read()
    if success:

        results_detection = yolo(img, classes=[0], conf=0.7, show=True, verbose=True)
        for detection in results_detection:
            boxes = detection.boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                centroid_x = ((x1 + x2)//2) - x1 
                centroid_y = ((y1 + y2)//2) - y1

                object_crop = img[y1:y2, x1:x2, :]
                results_segmentation = sam2(object_crop, points=[centroid_x, centroid_y], labels=[1])
                for segmentation in results_segmentation:
                    segmented_object = segmentation.plot() 
                    img[y1:y2, x1:x2] = segmented_object
        cv2.imshow('SAM2 + detection', img)
        out.write(img)

    else:
        break

cap.release()
cv2.destroyAllWindows()

# Experiment with box prompting

In [ ]:

cap = cv2.VideoCapture(path_to_video)
cap.set(3, 1280)
cap.set(4, 720)

output_video_path = "output_videos/processed_box_video.mp4"


fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))


while True:
    success, img = cap.read()
    if success:

        results_detection = yolo(img, classes=[0], conf=0.7, show=True, verbose=False)
        for detection in results_detection:
            boxes = detection.boxes
            boxes_list = []
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                boxes_list.append((x1, y1, x2, y2))
            results_segmentation = sam2(img, bboxes=boxes_list)
            for segmentation in results_segmentation:
                segmented_object = segmentation.plot()  
                img = cv2.addWeighted(img, 0.6, segmented_object, 0.4, 0)  

        cv2.imshow('SAM2 + detection', img)
        out.write(img)

        

    else:
        break

cap.release()
cv2.destroyAllWindows()